# IMPORTS

In [138]:
import numpy as np
import pandas as pd
import requests
import json
import time
from riotwatcher import LolWatcher, ApiError
import os
from tqdm.notebook import tqdm
import glob

In [2]:
import random
import cassiopeia as cass

## API KEY

In [181]:
API_KEY = 'RGAPI-60f0dc46-d361-44f4-8e6c-5f2a7d9b5952'

## PART 1: LEAGUE DOWNLOAD

In [169]:
def download_league(league_id):
    url = 'https://na1.api.riotgames.com/lol/league/v4/leagues/'
    response = requests.get(url + league_id + '?api_key=' + API_KEY)
    while response.status_code != 200:
        print('Sending new request...')
        time.sleep(1.25)
        response = requests.get(url + league_id + '?api_key=' + API_KEY)
    
    JSONContent = response.json()
    tier = JSONContent['tier']
    name = JSONContent['name']
    
    with open(f'league_data/{tier}_{name}.txt', 'w') as outfile:
        json.dump(JSONContent, outfile)
    
    print(f'Downloaded {tier}_{name} - DONE')

In [116]:
download_league('e6297c00-9648-49cc-8d25-30c0c6f9334a')

DONE


## PART 2: PLAYER DOWNLOADS

In [167]:
def download_player(player_name, league_name):
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
    time.sleep(1.25)
    response = requests.get(url + player_name + '?api_key=' + API_KEY)
    while response.status_code != 200:
        print('Sending new request...')
        time.sleep(1.25)
        response = requests.get(url + player_name + '?api_key=' + API_KEY)
    
    JSONContent = response.json()
    
    with open(f'summoner_data/{league_name}/{player_name}.txt', 'w') as outfile:
        json.dump(JSONContent, outfile)

In [168]:
def download_players(league_name):
    os.mkdir(f'summoner_data/{league_name}/')
    
    with open(f'league_data/{league_name}.txt') as json_file:
        data = json.load(json_file)
    
    summoner_names = []
    for summoner in data['entries']:
        summoner_name = summoner['summonerName']
        summoner_names.append(summoner_name)
        
    for player in tqdm(summoner_names):
        download_player(player, league_name)
    
    print('DONE')

In [132]:
download_players('GOLD_Darius\'s Runemasters')

Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...
Sending new request...

DONE


## PART 3: MATCHLIST DOWNLOAD

In [171]:
def download_matchlist(player_name, league_name):
    url = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/'
    with open(f'summoner_data/{league_name}/{player_name}.txt') as json_file:
        data = json.load(json_file)
    account_id = data['accountId']
    
    time.sleep(1.25)
    response = requests.get(url + account_id + '?queue=420' + '&api_key=' + API_KEY)
    while response.status_code != 200:
        print('Sending new request...')
        time.sleep(1.25)
        response = requests.get(url + account_id + '?queue=420' + '&api_key=' + API_KEY)
    
    JSONContent = response.json()
    
    with open(f'matchlist_data/{player_name}.txt', 'w') as outfile:
        json.dump(JSONContent, outfile)

In [172]:
def download_matchlists(league_name):
    account_files = glob.glob(f'summoner_data/{league_name}/*.txt')
    
    for account in tqdm(account_files):
        player_name = os.path.basename(account).split('.')[0]
        download_matchlist(player_name, league_name)
        
    print('DONE')

In [161]:
download_matchlists('GOLD_Darius\'s Runemasters')

Sending new request...

DONE


## PART 4: MATCH DOWNLOAD

In [178]:
def download_match(match_id):
    match_id_files = glob.glob(f'match_data/*.txt')
    if f'match_data/{match_id}.txt' in match_id_files:
        return
    
    url = 'https://na1.api.riotgames.com/lol/match/v4/matches/'
    
    time.sleep(1.25)
    response = requests.get(url + match_id + '?api_key=' + API_KEY)
    while response.status_code in [429, 500, 504]:
        print('Sending new request...')
        time.sleep(1.25)
        response = requests.get(url + match_id + '?api_key=' + API_KEY)
    
    if response.status_code != 200:
        print(f'API Error {response.status_code} - Dropping match {match_id}')
        return
    
    JSONContent = response.json()
    
    with open(f'match_data/{match_id}.txt', 'w') as outfile:
        json.dump(JSONContent, outfile)

In [164]:
def download_player_matches(player_name):
    with open(f'matchlist_data/{player_name}.txt') as json_file:
        data = json.load(json_file)
    match_ids = [str(match['gameId']) for match in data['matches']]
    
    for match_id in tqdm(match_ids, leave=False):
        download_match(match_id)

In [165]:
def download_players_matches():
    matchlist_files = glob.glob(f'matchlist_data/*.txt')
    
    for matchlist in tqdm(matchlist_files):
        player_name = os.path.basename(matchlist).split('.')[0]
        download_player_matches(player_name)
        
    print('DONE')

In [166]:
download_players_matches()

Sending new request...


Sending new request...


Sending new request...


Sending new request...


Sending new request...


...

Sending new request...
Sending new request...


Sending new request...


Sending new request...


Sending new request...



DONE


## Code to add Champion names from champion IDs

## Downloading another League from Silver Rank

In [170]:
download_league('79e28d48-cd14-461d-a715-37d4a19b3df9')

Downloaded SILVER_Jax's Dragons - DONE


In [173]:
download_players('SILVER_Jax\'s Dragons')


DONE


In [174]:
download_matchlists('SILVER_Jax\'s Dragons')

Sending new request...

DONE


In [180]:
download_players_matches()

API Error 404 - Dropping match 4380092144
API Error 404 - Dropping match 4379709565
API Error 404 - Dropping match 4379693478
API Error 404 - Dropping match 4379618245


Sending new request...


Sending new request...


Sending new request...
API Error 503 - Dropping match 3396787782
Sending new request...


Sending new request...


...

Sending new request...


Sending new request...


Sending new request...


Sending new request...



DONE


## Downloading another League from Silver Rank

In [182]:
download_league('72fbcbb0-c95f-4f04-a092-27f04540deb7')

Downloaded SILVER_Rumble's Urfriders - DONE


In [183]:
download_players('SILVER_Rumble\'s Urfriders')


DONE


In [184]:
download_matchlists('SILVER_Rumble\'s Urfriders')


DONE


In [185]:
download_players_matches()

Sending new request...


Sending new request...


Sending new request...


Sending new request...
Sending new request...


Sending new request...
Sending new request...


...

Sending new request...


Sending new request...


API Error 503 - Dropping match 3356579743
Sending new request...
Sending new request...


Sending new request...



DONE
